In [8]:
# TRAINING

# split into train and test set
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
import os
from Mask_RCNN.mrcnn.utils import Dataset
from Mask_RCNN.mrcnn.utils import extract_bboxes
from Mask_RCNN.mrcnn.visualize import display_instances
from numpy import expand_dims
from numpy import mean
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.utils import Dataset
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from matplotlib import pyplot
%matplotlib inline

from pycocotools.coco import COCO

# class that defines and loads the nightowls dataset
class OurDataset(Dataset):
    
    # load the dataset definitions
    def load_dataset(self, images_dir, annFile, is_train=True, val_percentage = 0.2):
        
        self.cocovar = COCO(annFile)
        self.imgIds = list(sorted(self.cocovar.imgs.keys()))
        self.images_dir = images_dir
        self.imgs_count = len(self.imgIds)
        
        self.add_class("dataset", 1, "pedestrian")
        
        i=0
        j = 0
        for im_id in self.imgIds:
            cocoimg = self.cocovar.loadImgs(ids=im_id)[0]
            img_filename = cocoimg['file_name']
            path = os.path.join(self.images_dir,img_filename)
            
            #load the ids of the annotations for the images
            image_annotations = self.cocovar.getAnnIds(im_id)
            
            pedestrian_present = False
            
            # for every annotation
            for ann_id in image_annotations:
                #load all the annotation data
                ann = self.cocovar.loadAnns(ids=ann_id)[0]
                
                if(ann['category_id'] == 1):
                    pedestrian_present = True
                
            # save only the images with pedestrians
            if j >= self.imgs_count*(1-val_percentage) and is_train == False and pedestrian_present == True: # validation
                self.add_image('dataset', image_id=i, path=path, real_id = im_id)
                i=i+1
            elif j < self.imgs_count*(1-val_percentage) and is_train == True and pedestrian_present == True: # training
                self.add_image('dataset', image_id=i, path=path, real_id = im_id)
                i=i+1
            
            j = j+1
            

    # load the masks for an image
    def load_mask(self, image_id):
        
        # get the image id and its filename
        #img_id = self.imgIds[image_id]
        img_id = self.image_info[image_id]['real_id']
        cocoimg = self.cocovar.loadImgs(ids=img_id)[0]
        
        img_filename = cocoimg['file_name']
        img_height = cocoimg['height']
        img_width = cocoimg['width']

        #load the ids of the annotations for the images
        image_annotations = self.cocovar.getAnnIds(img_id)
        
        boxes = []
        categories = []
        
        # for every annotation
        for ann_id in image_annotations:
            #load all the annotation data
            ann = self.cocovar.loadAnns(ids=ann_id)[0]
            
            # skip the non-pedestrinas
            if(ann['category_id'] != 1):
                continue
            
            data = []
            for i in ann['bbox']:
                data.append(i)
            categories.append(ann['category_id'])
            data[2] = data[2]+data[0] # from x,y,width,height to xmin,ymin,xmax,ymax
            data[3] = data[3]+data[1]
            boxes.append(data)
        
        # create one array for all masks, each on a different channel
        masks = zeros([img_height, img_width, len(boxes)], dtype='uint8')
        # create masks
        class_ids = list()
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            if categories[i] == 1:
                class_ids.append(self.class_names.index('pedestrian'))
            elif categories[i] == 2:
                class_ids.append(self.class_names.index('bycicle'))
            else:
                class_ids.append(self.class_names.index('ignore'))
            
        return masks, asarray(class_ids, dtype='int32')
               

    # load an image reference
    def image_reference(self, image_id):
        '''
        img_id = self.imgIds[image_id]
        cocoimg = self.cocovar.loadImgs(ids=img_id)[0]
        
        img_filename = cocoimg['file_name']
        return img_filename
        '''
        info = self.image_info[image_id]
        return info['path']

#ann_file_path = '/home/test/data/nightowls/nightowls_validation_small.json'
ann_file_path = '/home/test/data/nightowls/nightowls_training.json'
images_path = '/home/test/data/nightowls/validation/nightowls_validation'
    
# train set
train_set = OurDataset()
train_set.load_dataset(images_path, ann_file_path, is_train=True, val_percentage = 0.)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

# val set
val_set = OurDataset()
val_set.load_dataset(images_path, ann_file_path, is_train=False, val_percentage = 0.)
val_set.prepare()
print('Validation: %d' % len(val_set.image_ids))


'''
# print all the images with the masks in the dataset
for i in range(len(train_set.image_ids)):
    # define image id
    image_id = i
    # load the image
    image = train_set.load_image(image_id)
    # load the masks and the class ids
    mask, class_ids = train_set.load_mask(image_id)
    # extract bounding boxes from the masks
    bbox = extract_bboxes(mask)
    # display image with masks and bounding boxes
    info = train_set.image_info[image_id]
    print('Image id in the dataset: ' + str(image_id) + ' real_id: ' + str(info['real_id']))
    display_instances(image, bbox, mask, class_ids, train_set.class_names)
'''


loading annotations into memory...
Done (t=1.21s)
creating index...
index created!
Train: 25670
loading annotations into memory...
Done (t=0.80s)
creating index...
index created!
Validation: 0


"\n# print all the images with the masks in the dataset\nfor i in range(len(train_set.image_ids)):\n    # define image id\n    image_id = i\n    # load the image\n    image = train_set.load_image(image_id)\n    # load the masks and the class ids\n    mask, class_ids = train_set.load_mask(image_id)\n    # extract bounding boxes from the masks\n    bbox = extract_bboxes(mask)\n    # display image with masks and bounding boxes\n    info = train_set.image_info[image_id]\n    print('Image id in the dataset: ' + str(image_id) + ' real_id: ' + str(info['real_id']))\n    display_instances(image, bbox, mask, class_ids, train_set.class_names)\n"

In [1]:
from our_dataset import OurDataset


ann_file_path = '/home/test/data/nightowls/nightowls_training.json'
images_path = '/home/test/data/nightowls/validation/nightowls_validation'
    
# train set
train_set = OurDataset()
train_set.load_dataset(images_path, ann_file_path, is_train=True, val_percentage = 0.)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

/home/test/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/test/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/test/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/test/anaconda

loading annotations into memory...
Done (t=0.87s)
creating index...
index created!
Train: 130064
